In [42]:
import pickle
import csv
import numpy as np
import pandas as pd
import tensorflow as tf


from utils.dataset import DataSet
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Import data

In [9]:
trainDataset = DataSet()
testDataset = DataSet("competition_test")

FullCorpus = []
tokenizedTrainHeadlines = []
tokenizedTrainBodies = []
tokenizedTestHeadlines = []
tokenizedTestBodies = []

#seen headlines and body tracker to ensure we don't have duplicates when building TF for corpus
trainHeadlinesSeen = {}
trainBodiesSeen = {}
testHeadlinesSeen = {}
testBodiesSeen = {}


for stance in trainDataset.stances:
    if stance['Headline'] not in trainHeadlinesSeen:
        tokenizedHeadline = word_tokenize(stance['Headline'])
        tokenizedTrainHeadlines.append(tokenizedHeadline)
        trainHeadlinesSeen[stance['Headline']] = tokenizedHeadline
    
    if stance['Body ID'] not in trainBodiesSeen:
        tokenizedBody = word_tokenize(trainDataset.articles[stance['Body ID']])
        tokenizedTrainBodies.append(tokenizedBody)
        trainBodiesSeen[stance['Body ID']] = tokenizedBody

for stance in testDataset.stances:
    if stance['Headline'] not in testHeadlinesSeen:
        tokenizedHeadline = word_tokenize(stance['Headline'])
        tokenizedTestHeadlines.append(tokenizedHeadline)
        testHeadlinesSeen[stance['Headline']] = tokenizedHeadline
    
    if stance['Body ID'] not in testBodiesSeen:
        tokenizedBody = word_tokenize(testDataset.articles[stance['Body ID']])
        tokenizedTestBodies.append(tokenizedBody)
        testBodiesSeen[stance['Body ID']] = tokenizedBody

Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset
Total stances: 25413
Total bodies: 904


## Build tokenizers and count vectorizers

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([' '.join(seq) for seq in tokenizedTrainHeadlines + tokenizedTrainBodies + ["<UNK>"]])

vocabulary=tokenizer.word_index
vocabulary=list(vocabulary.keys())

countVectorizer = CountVectorizer(vocabulary=vocabulary)
trainCorpusBagOfWords = countVectorizer.fit_transform([' '.join(seq) for seq in tokenizedTrainHeadlines + tokenizedTrainBodies])

tfVectorizer = TfidfVectorizer().fit([' '.join(seq) for seq in tokenizedTrainHeadlines + tokenizedTrainBodies + ['<UNK>']]) 


## Build feature and label vectors

In [11]:
trainFeatures = []
trainLabels = []

testFeatures = []
testLabels = []

# 0=unrelated 1=discuss 2=agree 3=disagree
for stance in trainDataset.stances:
    label = [1,0,0,0] if stance['Stance'] == 'unrelated' else [0,1,0,0] if stance['Stance'] == 'discuss' else [0,0,1,0] if stance['Stance'] == 'agree' else [0,0,0,1]  
    trainLabels.append(label)
    headline = [' '.join(trainHeadlinesSeen[stance['Headline']])]
    body = [' '.join(trainBodiesSeen[stance['Body ID']])]
    headlineTermVec = list(countVectorizer.transform(headline).toarray())[0].reshape(1, -1)
    bodyTermVec = list(countVectorizer.transform(body).toarray())[0].reshape(1, -1)
    
    tfidfHeadline = tfVectorizer.transform(headline).toarray()
    tfidfBody = tfVectorizer.transform(body).toarray()
    tfidf_cos = cosine_similarity(tfidfHeadline, tfidfBody)[0].reshape(1, 1)
    x = np.hstack(( tfidfHeadline, tfidfBody,tfidf_cos )).ravel()
    trainFeatures.append(x)
    
for stance in testDataset.stances:
    label = [1,0,0,0] if stance['Stance'] == 'unrelated' else [0,1,0,0] if stance['Stance'] == 'discuss' else [0,0,1,0] if stance['Stance'] == 'agree' else [0,0,0,1] 
    testLabels.append(label)
    headline = [' '.join(testHeadlinesSeen[stance['Headline']])]
    body = [' '.join(testBodiesSeen[stance['Body ID']])]
    headlineTermVec = list(countVectorizer.transform(headline).toarray())[0].reshape(1, -1)
    bodyTermVec = list(countVectorizer.transform(body).toarray())[0].reshape(1, -1)
    
    tfidfHeadline = tfVectorizer.transform(headline).toarray()
    tfidfBody = tfVectorizer.transform(body).toarray()
    tfidf_cos = cosine_similarity(tfidfHeadline, tfidfBody)[0].reshape(1, 1)
    x = np.hstack(( tfidfHeadline, tfidfBody,tfidf_cos )).ravel()
    testFeatures.append(x)


In [12]:
trainFeatures = np.array(trainFeatures)
testFeatures = np.array(testFeatures)

In [13]:
trainLabels = np.array(trainLabels)
testLabels = np.array(testLabels)

In [14]:
testFeatures.shape

(25413, 46675)

In [15]:
trainFeatures.shape

(49972, 46675)

# Build Models

In [16]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional


#INPUT_DIM = 2*len(tokenizer.word_index) + 1
INPUT_DIM = trainFeatures.shape[1]
BATCH_SIZE = 512
N_EPOCHS = 10

## Sigmoid Model

In [20]:
modelSig = Sequential()
modelSig.add(Dense(256, activation='sigmoid', input_dim=INPUT_DIM, name="activation_layer"))
modelSig.add(Dense(4, activation='softmax', name='output_layer'))
modelSig.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 activation_layer (Dense)    (None, 256)               11949056  
                                                                 
 output_layer (Dense)        (None, 4)                 1028      
                                                                 
Total params: 11,950,084
Trainable params: 11,950,084
Non-trainable params: 0
_________________________________________________________________


In [21]:
modelSig.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
modelSig.fit(trainFeatures, trainLabels,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          validation_data=(testFeatures, testLabels))

Epoch 1/10
98/98 [==============================] - 28s 277ms/step - loss: 0.7147 - accuracy: 0.7376 - val_loss: 0.8014 - val_accuracy: 0.7220
Epoch 2/10
98/98 [==============================] - 18s 188ms/step - loss: 0.5709 - accuracy: 0.7819 - val_loss: 0.7471 - val_accuracy: 0.7303
Epoch 3/10
98/98 [==============================] - 18s 179ms/step - loss: 0.4651 - accuracy: 0.8313 - val_loss: 0.6979 - val_accuracy: 0.7586
Epoch 4/10
98/98 [==============================] - 18s 179ms/step - loss: 0.3804 - accuracy: 0.8690 - val_loss: 0.6489 - val_accuracy: 0.7847
Epoch 5/10
98/98 [==============================] - 17s 178ms/step - loss: 0.3136 - accuracy: 0.8973 - val_loss: 0.6115 - val_accuracy: 0.8112
Epoch 6/10
98/98 [==============================] - 19s 194ms/step - loss: 0.2616 - accuracy: 0.9169 - val_loss: 0.5777 - val_accuracy: 0.8298
Epoch 7/10
98/98 [==============================] - 19s 197ms/step - loss: 0.2195 - accuracy: 0.9332 - val_loss: 0.5674 - val_accuracy: 0.8314

In [24]:
score, acc = modelSig.evaluate(testFeatures, testLabels, batch_size=BATCH_SIZE)
print(acc)

50/50 [==============================] - 4s 68ms/step - loss: 0.5043 - accuracy: 0.8463
0.8462991118431091


## Relu Model

In [17]:
modelRelu = Sequential()
modelRelu.add(Dense(256, activation='relu', input_dim=INPUT_DIM, name="activation_layer"))
modelRelu.add(Dense(4, activation='softmax', name='output_layer'))
modelRelu.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 activation_layer (Dense)    (None, 256)               11949056  
                                                                 
 output_layer (Dense)        (None, 4)                 1028      
                                                                 
Total params: 11,950,084
Trainable params: 11,950,084
Non-trainable params: 0
_________________________________________________________________


2022-03-30 14:19:37.309706: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
modelRelu.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
modelRelu.fit(trainFeatures, trainLabels,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          validation_data=(testFeatures, testLabels))

Epoch 1/10
98/98 [==============================] - 34s 335ms/step - loss: 0.6617 - accuracy: 0.7730 - val_loss: 0.7163 - val_accuracy: 0.7663
Epoch 2/10
98/98 [==============================] - 23s 229ms/step - loss: 0.3094 - accuracy: 0.8987 - val_loss: 0.6026 - val_accuracy: 0.8216
Epoch 3/10
98/98 [==============================] - 21s 209ms/step - loss: 0.1592 - accuracy: 0.9549 - val_loss: 0.5427 - val_accuracy: 0.8285
Epoch 4/10
98/98 [==============================] - 23s 234ms/step - loss: 0.0971 - accuracy: 0.9753 - val_loss: 0.4959 - val_accuracy: 0.8397
Epoch 5/10
98/98 [==============================] - 23s 237ms/step - loss: 0.0670 - accuracy: 0.9842 - val_loss: 0.4856 - val_accuracy: 0.8454
Epoch 6/10
98/98 [==============================] - 23s 234ms/step - loss: 0.0504 - accuracy: 0.9885 - val_loss: 0.4795 - val_accuracy: 0.8479
Epoch 7/10
98/98 [==============================] - 23s 234ms/step - loss: 0.0403 - accuracy: 0.9904 - val_loss: 0.5022 - val_accuracy: 0.8447

In [20]:
score, acc = modelRelu.evaluate(testFeatures, testLabels, batch_size=BATCH_SIZE)
print(acc)

50/50 [==============================] - 5s 75ms/step - loss: 0.5345 - accuracy: 0.8486
0.8485814332962036


## Tanh Model

In [29]:
modelTanh = Sequential()
modelTanh.add(Dense(256, activation='tanh', input_dim=INPUT_DIM, name="activation_layer"))
modelTanh.add(Dense(4, activation='softmax', name='output_layer'))
modelTanh.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 activation_layer (Dense)    (None, 256)               11949056  
                                                                 
 output_layer (Dense)        (None, 4)                 1028      
                                                                 
Total params: 11,950,084
Trainable params: 11,950,084
Non-trainable params: 0
_________________________________________________________________


In [30]:
modelTanh.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
modelTanh.fit(trainFeatures, trainLabels,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          validation_data=(testFeatures, testLabels))

Epoch 1/10
98/98 [==============================] - 28s 272ms/step - loss: 0.6108 - accuracy: 0.7860 - val_loss: 0.7108 - val_accuracy: 0.7683
Epoch 2/10
98/98 [==============================] - 18s 185ms/step - loss: 0.3077 - accuracy: 0.8955 - val_loss: 0.5923 - val_accuracy: 0.8184
Epoch 3/10
98/98 [==============================] - 19s 190ms/step - loss: 0.1825 - accuracy: 0.9430 - val_loss: 0.5590 - val_accuracy: 0.8184
Epoch 4/10
98/98 [==============================] - 18s 180ms/step - loss: 0.1233 - accuracy: 0.9643 - val_loss: 0.5466 - val_accuracy: 0.8196
Epoch 5/10
98/98 [==============================] - 18s 184ms/step - loss: 0.0948 - accuracy: 0.9733 - val_loss: 0.5551 - val_accuracy: 0.8182
Epoch 6/10
98/98 [==============================] - 18s 181ms/step - loss: 0.0779 - accuracy: 0.9773 - val_loss: 0.5639 - val_accuracy: 0.8172
Epoch 7/10
98/98 [==============================] - 18s 188ms/step - loss: 0.0683 - accuracy: 0.9795 - val_loss: 0.5581 - val_accuracy: 0.8297

In [32]:
score, acc = modelTanh.evaluate(testFeatures, testLabels, batch_size=BATCH_SIZE)
print(acc)

50/50 [==============================] - 5s 73ms/step - loss: 0.5870 - accuracy: 0.8360
0.8359894752502441


## Relu Experiments

Since the last 3 models showed Relu as the top performer it will be used in the final model, after experimenting with dropout rates and normalizations

In [77]:
modelRelu2 = Sequential()
modelRelu2.add(Dropout(rate=0.1, name='dropout_1'))
modelRelu2.add(BatchNormalization(name='bn'))
modelRelu2.add(Dense(256, activation='relu', input_dim=INPUT_DIM, name="activation_layer"))
modelRelu2.add(Dense(4, activation='softmax', name='output_layer'))

In [78]:
modelRelu2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [79]:
modelRelu2.fit(trainFeatures, trainLabels,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          validation_data=(testFeatures, testLabels))

Epoch 1/10
98/98 [==============================] - 55s 544ms/step - loss: 0.3740 - accuracy: 0.8700 - val_loss: 0.8803 - val_accuracy: 0.7235
Epoch 2/10
98/98 [==============================] - 43s 434ms/step - loss: 0.0814 - accuracy: 0.9735 - val_loss: 0.9780 - val_accuracy: 0.7232
Epoch 3/10
98/98 [==============================] - 42s 426ms/step - loss: 0.0411 - accuracy: 0.9864 - val_loss: 0.8768 - val_accuracy: 0.8002
Epoch 4/10
98/98 [==============================] - 42s 427ms/step - loss: 0.0255 - accuracy: 0.9916 - val_loss: 0.9336 - val_accuracy: 0.6194
Epoch 5/10
98/98 [==============================] - 45s 461ms/step - loss: 0.0198 - accuracy: 0.9940 - val_loss: 0.7520 - val_accuracy: 0.7847
Epoch 6/10
98/98 [==============================] - 44s 451ms/step - loss: 0.0161 - accuracy: 0.9950 - val_loss: 0.7227 - val_accuracy: 0.7592
Epoch 7/10
98/98 [==============================] - 46s 475ms/step - loss: 0.0158 - accuracy: 0.9951 - val_loss: 0.7313 - val_accuracy: 0.7374

In [80]:
score, acc = modelRelu2.evaluate(testFeatures, testLabels, batch_size=BATCH_SIZE)
print(acc)

50/50 [==============================] - 6s 99ms/step - loss: 0.7540 - accuracy: 0.7352
0.7352142333984375


# Build Results csv for Relu model

Load model

In [22]:
from keras.models import load_model

In [23]:
model = load_model("FinalRelu.model")

In [27]:
predicted = model.predict(testFeatures)

In [39]:
result = {"Headline": [], "Body ID": [], "Stance": []}

# 0=unrelated 1=discuss 2=agree 3=disagree
for i, stance in enumerate(testDataset.stances):
    result["Headline"].append(stance['Headline'])
    result["Body ID"].append(stance['Body ID'])
    prediction = predicted[i]
    predictionIndex = np.where(prediction == np.amax(prediction))[0][0]
    stance = 'unrelated' if predictionIndex == 0 else 'discuss' if predictionIndex == 1 else 'agree' if predictionIndex == 2 else 'disagree'
    result["Stance"].append(stance)

In [43]:
result = pd.DataFrame.from_dict(result)
    
result.to_csv('answer.csv', index=False, encoding='utf-8')